In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install huggingface_hub transformers datasets gradio pipreqs TextBlob emot xformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

In [4]:
!pip install accelerate>=0.20.1
!pip install transformers[torch] accelerate -U

In [41]:
!huggingface-cli login --token 

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [42]:
from huggingface_hub import create_repo
create_repo("Orseer/test-model_ROBERTA")
'https://huggingface.co/Orseer/test-model_ROBERTA'

'https://huggingface.co/Orseer/test-model_ROBERTA'

In [6]:
# Import libraries
import os
import uuid
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from scipy.special import softmax
import gradio as gr

from google.colab import drive
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import IntervalStrategy
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback
from transformers import pipeline
from transformers import TrainingArguments
from transformers import Trainer
from torch import nn




In [7]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [8]:
# Load the dataset and display some values

# Load the CSV file into a DataFrame

url = "https://github.com/Azubi-Africa/Career_Accelerator_P5-NLP/raw/master/zindi_challenge/data/Train.csv"

df = pd.read_csv(url)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [10]:
# Select rows with missing values
df.isnull().sum()

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

In [11]:
# Select rows with missing values
df[df.isnull().any(axis=1)]

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some pare...,1,0.666667,NaN


In [12]:
# Extract complete text from 'safe_text' column
complete_text = df.iloc[4798]['safe_text']
complete_text

'#lawandorderSVU '

In [13]:
# Select row by index and assign values to columns
df.loc[4798, 'label'] = 0
df.loc[4798, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[4798, df.columns.get_loc('safe_text')] = complete_text


In [14]:
# Generate random UUID string for tweet_id
'''UUIDs are often used in software applications for various purposes such as generating unique IDs for entities,
tracking unique user sessions, or creating unique file names'''
rand_tweet_id = str(uuid.uuid4())

# Select row by index and assign values to columns
row_index = 4799
df.loc[row_index, 'tweet_id'] = rand_tweet_id
df.loc[row_index, 'label'] = 1
df.loc[row_index, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[row_index, df.columns.get_loc('safe_text')] = df.iloc[row_index, 1]


In [15]:
df[df.duplicated()].sum()

tweet_id     0.0
safe_text    0.0
label        0.0
agreement    0.0
dtype: float64

### Handling Imbalance Data

In [16]:
df['label'].value_counts()

 0.0    4909
 1.0    4054
-1.0    1038
Name: label, dtype: int64

In [17]:
# Find the maximum count among all classes
max_class_count = df['label'].value_counts().max()

# Group the dataframe by 'label'
grouped = df.groupby('label')

# Sample each group to match the max_class_count
balanced_df = grouped.apply(lambda x: x.sample(max_class_count, replace=True)).reset_index(drop=True)

# The 'balanced_df' now contains an equal number of instances for each class
print(balanced_df['label'].value_counts())


-1.0    4909
 0.0    4909
 1.0    4909
Name: label, dtype: int64


In [18]:
# Split the train data => {train, eval}
train, eval = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11781 entries, 2435 to 7270
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   11781 non-null  object 
 1   safe_text  11781 non-null  object 
 2   label      11781 non-null  float64
 3   agreement  11781 non-null  float64
dtypes: float64(2), object(2)
memory usage: 460.2+ KB


In [20]:
eval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2946 entries, 13730 to 9863
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   2946 non-null   object 
 1   safe_text  2946 non-null   object 
 2   label      2946 non-null   float64
 3   agreement  2946 non-null   float64
dtypes: float64(2), object(2)
memory usage: 115.1+ KB


In [21]:
# Save splitted subsets

# Define file path

file_path = '/content/drive/MyDrive/temporal/LP5 Team Quebec'

#"/content/drive/MyDrive/NLP2_/new M2"

train.to_csv(os.path.join(file_path, "train_subset.csv"), index=False)
eval.to_csv(os.path.join(file_path, "eval_subset.csv"), index=False)

In [22]:
# Load the CSV files into a dataset

dataset = load_dataset('csv', data_files={
    'train': '/content/drive/MyDrive/temporal/LP5 Team Quebec/model/train_subset.csv',
    'eval': '/content/drive/MyDrive/temporal/LP5 Team Quebec/eval_subset.csv'
}, encoding='ISO-8859-1')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
#use hub_model_id="finetuned-Sentiment-classfication-ROBERTA-model
#use hub_model_id="finetuned-Sentiment-classfication-BERT-model
#use hub_model_id="finetuned-Sentiment-classfication-DISTILBERT-model
#use hub_model_id="finetuned-Sentiment-classfication-XLM-model

In [23]:
torch.cuda.is_available()

True

In [51]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',                          # Directory where the model checkpoints and evaluation results will be stored
    evaluation_strategy=IntervalStrategy.STEPS,      # Interval for evaluating the model during training (every specified number of steps)
    save_strategy=IntervalStrategy.STEPS,            # Interval for saving the model during training (every specified number of steps)
    save_steps=500,                                  # Number of steps between two saves
    load_best_model_at_end=True,                     # Whether to load the best model at the end of training
    num_train_epochs=16,                              # Number of training epochs
    per_device_train_batch_size=4,                   # Batch size per GPU for training
    per_device_eval_batch_size=4,                    # Batch size per GPU for evaluation
    learning_rate=3e-5,                              # Learning rate
    weight_decay=0.01,                               # Weight decay
    warmup_steps=500,                                # Number of warmup steps
    logging_steps=500,                               # Number of steps between two logs
    fp16=True,                                       # Whether to use 16-bit precision
    gradient_accumulation_steps=16,                  # Number of steps to accumulate gradients before performing an optimizer step
    dataloader_num_workers=2,                        # Number of workers to use for loading data
    push_to_hub=True,                                # Whether to push the model checkpoints to the Hugging Face hub
    hub_model_id="Orseer/test-model_ROBERTA",  # Model ID to use when pushing the model to the Hugging Face hub
)



# Define the early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,                       # Number of epochs with no improvement before stopping training
    early_stopping_threshold=0.01,                   # Minimum improvement in the metric for considering an improvement
)

# Combine the training arguments and the early stopping callback
training_args.callbacks = [early_stopping]


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [52]:
tokenizer_ROBERTA = AutoTokenizer.from_pretrained('roberta-base')
'''
This code instantiates a tokenizer for the BERT (Bidirectional Encoder Representations from Transformers)
pre-trained model with the bert-base-cased configuration.

'''


'\nThis code instantiates a tokenizer for the BERT (Bidirectional Encoder Representations from Transformers)\npre-trained model with the bert-base-cased configuration.\n\n'

In [53]:
# Define a function to transform the label values
def transform_labels(label):
    # Extract the label value
    label = label['label']
    # Map the label value to an integer value
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2
    # Return a dictionary with a single key-value pair
    return {'labels': num}

# Define a function to tokenize the text data
def tokenize_data(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer_ROBERTA(example['safe_text'], padding='max_length')

# Apply the transformation functions to the dataset using the 'map' method
# This transforms the label values and tokenizes the text data
dataset_out = dataset.map(transform_labels)

dataset_ROBERTA = dataset_out.map(tokenize_data, batched=True)

# Define a list of column names to remove from the dataset
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

# Apply the 'transform_labels' function to the dataset to transform the label values
# Also remove the columns specified in 'remove_columns'

dataset_ROBERTA = dataset_ROBERTA.map(transform_labels, remove_columns=remove_columns)

In [54]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model_ROBERTA = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
train_dataset_ROBERTA = dataset_ROBERTA['train'].shuffle(seed=10) #.select(range(40000)) # to select a part


In [56]:
eval_dataset_ROBERTA = dataset_ROBERTA['eval'].shuffle(seed=10)


In [57]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    rmse = np.sqrt(np.mean((predictions - labels)**2))
    return {"rmse": rmse}


In [58]:
trainer_ROBERTA = Trainer(
    model=model_ROBERTA,
    args=training_args,
    train_dataset=train_dataset_ROBERTA,
    eval_dataset=eval_dataset_ROBERTA,
    compute_metrics=compute_metrics    # Add this line to define the compute_metrics function
)

Cloning https://huggingface.co/Orseer/test-model_ROBERTA into local empty directory.


In [59]:
trainer_ROBERTA.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rmse
500,0.675500,0.413641,0.512571
1000,0.190900,0.273277,0.342706
1500,0.078400,0.317347,0.324388
2000,0.038200,0.352283,0.303856


Step,Training Loss,Validation Loss,Rmse
500,0.675500,0.413641,0.512571
1000,0.190900,0.273277,0.342706
1500,0.078400,0.317347,0.324388
2000,0.038200,0.352283,0.303856
2500,0.021300,0.394661,0.286610


TrainOutput(global_step=2944, training_loss=0.1725694235900174, metrics={'train_runtime': 7586.3638, 'train_samples_per_second': 24.847, 'train_steps_per_second': 0.388, 'total_flos': 4.956293761084109e+16, 'train_loss': 0.1725694235900174, 'epoch': 15.99})

In [60]:
# Evaluate the model
eval_results = trainer_ROBERTA.evaluate()

# Create a dictionary of the evaluation results
results_dict = {
    "Model": "roberta-base",
    "Loss": eval_results["eval_loss"],
    "RMSE": eval_results["eval_rmse"],
    "Runtime": eval_results["eval_runtime"],
    "Samples Per Second": eval_results["eval_samples_per_second"],
    "Steps Per Second": eval_results["eval_steps_per_second"],
    "Epoch": eval_results["epoch"]
}

# Create a pandas DataFrame from the dictionary
results_df = pd.DataFrame([results_dict])

# Sort the results by "eval_rmse" in ascending order and get the name and state dict of the best model
best_model = results_df.loc[results_df['RMSE'].idxmin()]

print(best_model)


Model                 roberta-base
Loss                      0.273277
RMSE                      0.342706
Runtime                    41.8003
Samples Per Second          70.478
Steps Per Second            17.631
Epoch                        15.99
Name: 0, dtype: object


In [61]:
 # Push the final fine-tuned model to the Hugging Face model hub

trainer_ROBERTA.push_to_hub("Orseer/finetuned-Sentiment-classfication-ROBERTA-model")

Upload file runs/Aug02_09-12-48_5d8050df9a9a/events.out.tfevents.1690967662.5d8050df9a9a.508.0:   0%|         …

Upload file runs/Aug02_09-12-48_5d8050df9a9a/events.out.tfevents.1690975330.5d8050df9a9a.508.1:   0%|         …

To https://huggingface.co/Orseer/test-model_ROBERTA
   8505baa..61aa352  main -> main

   8505baa..61aa352  main -> main

To https://huggingface.co/Orseer/test-model_ROBERTA
   61aa352..f26a565  main -> main

   61aa352..f26a565  main -> main



'https://huggingface.co/Orseer/test-model_ROBERTA/commit/61aa35251d9a8865dafc4b9e416485178d55e317'

In [62]:
tokenizer_ROBERTA.push_to_hub("Orseer/finetuned-Sentiment-classfication-ROBERTA-model")

CommitInfo(commit_url='https://huggingface.co/Orseer/finetuned-Sentiment-classfication-ROBERTA-model/commit/d2dc10b2adc25176023cd0cead84ee4775819b80', commit_message='Upload tokenizer', commit_description='', oid='d2dc10b2adc25176023cd0cead84ee4775819b80', pr_url=None, pr_revision=None, pr_num=None)

In [63]:
model_ROBERTA.push_to_hub("Orseer/finetuned-Sentiment-classfication-ROBERTA-model")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Orseer/finetuned-Sentiment-classfication-ROBERTA-model/commit/4d6f10758f3f4f2babce23833e704d8538fbb8d4', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='4d6f10758f3f4f2babce23833e704d8538fbb8d4', pr_url=None, pr_revision=None, pr_num=None)

In [64]:
# Load the tokenizer
tokenizer = tokenizer_ROBERTA.from_pretrained("Orseer/finetuned-Sentiment-classfication-ROBERTA-model")

# Load the fine-tuned model
model = pipeline("text-classification", model="Orseer/finetuned-Sentiment-classfication-ROBERTA-model", tokenizer=tokenizer)


In [65]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Make predictions on some example text
result = model("I love these covid vaccines.")

# Map the numerical label to the corresponding class name
result[0]["label"] = label_map[int(result[0]["label"].split("_")[1])]

# Print the predicted label and score
print(result)

[{'label': 'positive', 'score': 0.9968825578689575}]


In [ ]:
#!pip freeze > /content/drive/MyDrive/NLP_2/new_M2/requirements.txt

In [ ]:
#!ls {file_path}

ls: cannot access '2/new': No such file or directory
ls: cannot access 'M2': No such file or directory
/content/drive/MyDrive/NLP
